# Isolation Levels

## Read Committed Isolation Level

### Specifications

- Select Queries: sees only data committed before the **query** began, easy to achieve with MVCC snapshot.
- DML Queries: blocked by other DML queries on the same rows, wait for them to commit / rollback.\
If they succeed the updated version of the rows that were found are evaluated again against the `WHERE` qualifiers.

### Illustration

#### Simple Usage - May Be OK

Transfer between accounts
```SQL
BEGIN;
UPDATE accounts SET balance = balance + 100.00 WHERE acctnum = 12345;
-- run from another session: UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 12345;
UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 7534;
COMMIT;
```

#### Complex Usage - Problematic

Assume website is a two-row table with website.hits equaling 9 and 10:

```SQL
BEGIN;
UPDATE website SET hits = hits + 1; -- Which rows are updated??
-- run from another session:  DELETE FROM website WHERE hits = 10;
COMMIT;
```

## Repeatable Read Isolation Level

### Specifications

- Classic snapshot isolation.
- Select Queries: sees only data committed before the **first non-transaction-control statement in the transaction** began.
- DML Queries: current transaction is blocked by other DML transactions on the same rows, wait for them to commit / rollback.\
If they succeed the, the transaction is rolled back.
- An app that uses this or more strict level should be prepare to deal with serialization failures -> retry transaction.

### Illustration

```SQL
BEGIN;
SELECT * FROM website WHERE hits = 10;
-- run from another session:  DELETE FROM website WHERE hits = 10;
UPDATE website SET hits = hits + 1; -- Would crash with failure!
COMMIT;
```

`ERROR:  could not serialize access due to concurrent update`

## Serializable Isolation Level

- Serializable snapshot isolation.
- Similar to repeatable read, but takes predicate (non-blocking) locks to identify potential read-write dependencies.
- Eliminates any non-serial anomalies, *assuming that all the transactions operating on this data are serializable!*